In [ ]:
# Libraries

In [ ]:
import os
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras.losses import Loss
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import (Input, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, BatchNormalization, 
                                     ReLU, Add, Dropout, LSTM, MultiHeadAttention, Reshape,Flatten)
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    confusion_matrix, roc_curve, precision_recall_curve, classification_report
)
import tensorflow as tf
import shap
from lime import lime_image
from skimage.segmentation import mark_boundaries
from tf_explain.core.grad_cam import GradCAM
import json

In [ ]:
# Data Loading and Preprocessing

In [ ]:
root_dir = r'D:\Multi-Class Diabetic Retinopathy Classification\data\aptos2019-blindness-detection'
train_dir = os.path.join(root_dir, "train")
test_dir = os.path.join(root_dir, "test")

In [ ]:
# Image preprocessing and augmentation
def preprocess_image(image):

    h, w, _ = image.shape
    center_x, center_y = w // 2, h // 2
    crop_size = min(center_x, center_y)  
    cropped_image = image[
        center_y - crop_size:center_y + crop_size,
        center_x - crop_size:center_x + crop_size
    ]

    resized_image = cv2.resize(cropped_image, (512, 512))

    normalized_image = resized_image / 255.0  
    return normalized_image

train_datagen = ImageDataGenerator(
    horizontal_flip=True,
    rotation_range=90,
    zoom_range=0.2,
    brightness_range=[0.8, 1.2],
    preprocessing_function=preprocess_image, 
    validation_split=0.2               
)
test_datagen = ImageDataGenerator(rescale=1.0/255)

In [ ]:
# Load and preprocess the training and testing images
batch_size = 8  # 统一 batch_size
train_generator = train_datagen.flow_from_directory(
    train_dir, target_size=(512, 512), batch_size=batch_size, class_mode='categorical', subset='training', shuffle=True  
)
validation_generator = train_datagen.flow_from_directory(
    train_dir, target_size=(512, 512), batch_size=batch_size, class_mode='categorical', subset='validation', shuffle=False
)
test_generator = test_datagen.flow_from_directory(
    test_dir, target_size=(512, 512), batch_size=batch_size, class_mode='categorical', shuffle=False
)
n_classes = len(train_generator.class_indices)

In [ ]:
# Model Definition

In [ ]:
def residual_block(x, filters, strides=1):
    shortcut = x
    if strides != 1 or x.shape[-1] != filters * 4:
        shortcut = Conv2D(filters * 4, kernel_size=(1, 1), strides=strides, padding='same',
                          kernel_initializer='he_normal', kernel_regularizer=l2(0.001))(shortcut)
        shortcut = BatchNormalization()(shortcut)

    # 1x1 Conv (Reduce dimension)
    x = Conv2D(filters, kernel_size=(1, 1), strides=strides, padding='same', 
               kernel_initializer='he_normal', kernel_regularizer=l2(0.001))(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # 3x3 Conv
    x = Conv2D(filters, kernel_size=(3, 3), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(0.001))(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # 1x1 Conv (Restore dimension)
    x = Conv2D(filters * 4, kernel_size=(1, 1), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(0.001))(x)
    x = BatchNormalization()(x)

    # Add shortcut to the main path
    x = Add()([shortcut, x])
    x = ReLU()(x)
    return x


In [ ]:
def build_model(input_shape=(512, 512, 3), n_classes=n_classes):
    input_layer = Input(shape=input_shape)

    x = Conv2D(32, (7, 7), strides=(2, 2), padding='same')(input_layer)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    x = residual_block(x, 32)
    x = residual_block(x, 32)

    x = residual_block(x, 64, strides=2)
    x = residual_block(x, 64)

    # 🔹 Multi-Head Attention
    x = Reshape((-1, x.shape[-1]))(x)
    x = MultiHeadAttention(num_heads=2, key_dim=8)(x, x)  
    x = Reshape((128, 128, 64))(x)  

    x = residual_block(x, 128, strides=2)
    x = residual_block(x, 128)

    x = residual_block(x, 256, strides=2)
    x = residual_block(x, 256)

    x = GlobalAveragePooling2D()(x)

    # 🔹 修正 LSTM 结构
    x = Reshape((1, -1))(x)
    x = LSTM(32, return_sequences=False)(x)

    x = Dense(2048, activation='relu', kernel_regularizer=l2(0.001))(x)
    x = Dropout(0.5)(x)

    output = Dense(n_classes, activation='softmax')(x)

    model = Model(inputs=input_layer, outputs=output)
    return model

In [ ]:
# Model Training

In [ ]:
# 6️⃣ 训练策略
def lr_scheduler(epoch, lr):
    if epoch < 5:
        return 1e-3  
    elif epoch < 20:
        return 1e-4
    else:
        return 1e-5  

# 7️⃣ 训练模型
model = build_model()

# 🔹 自定义损失函数
class CustomLoss(Loss):
    def call(self, y_true, y_pred):
        return K.categorical_crossentropy(y_true, y_pred)

custom_loss = CustomLoss()

model.compile(optimizer=Adam(learning_rate=0.0002), loss=custom_loss, metrics=['accuracy'])


In [ ]:
# 🔹 训练回调
callbacks = [
    ModelCheckpoint('model_best.h5', monitor='val_loss', save_best_only=True, mode='min'),
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=0.00002, mode='min'),
    CSVLogger('training_log.csv', append=True),
    tf.keras.callbacks.LearningRateScheduler(lr_scheduler)
]
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=50,
    steps_per_epoch=len(train_generator),
    validation_steps=len(validation_generator),
    callbacks=callbacks,
    workers=6,
    max_queue_size=30
)


In [ ]:
# Model Evaluation

In [ ]:
save_dir='plots'
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

In [ ]:
# Plot the training and validation loss and accuracy curves
plt.figure(figsize=(12, 5))
# Accuracy curves
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(save_dir, 'accuracy_plot.png'), dpi=600, bbox_inches='tight')
plt.show()
# Plot the training and validation loss and accuracy curves
plt.figure(figsize=(12, 5))
# Loss curves
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(save_dir, 'loss_plot.png'), dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
# ✅ 加载最优模型
model = load_model('model_best.h5', custom_objects={'CustomLoss': CustomLoss()})

# ✅ 进行预测
predictions = model.predict(test_generator)
true_labels = test_generator.classes
class_labels = list(test_generator.class_indices.keys())
predicted_classes = np.argmax(predictions, axis=1)

# ✅ 计算混淆矩阵
cm = confusion_matrix(true_labels, predicted_classes)

# ✅ 评估模型
def evaluate_model(true_labels, predicted_classes, class_labels, save_dir):
    # 确保保存目录存在
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # 计算准确率
    micro_accuracy = accuracy_score(true_labels, predicted_classes)

    # 生成分类报告
    classification_report_dict = classification_report(
        true_labels, predicted_classes, target_names=class_labels, output_dict=True, zero_division=0
    )

    # 计算每个类别的 Specificity 和 Accuracy
    specificity_per_class = []
    accuracy_per_class = []
    
    for i in range(len(class_labels)):
        TP = cm[i, i]  # 真实正例
        FN = np.sum(cm[i, :]) - TP  # 假阴性
        FP = np.sum(cm[:, i]) - TP  # 假阳性
        TN = np.sum(cm) - (TP + FN + FP)  # 真实负例

        specificity = TN / (TN + FP) if (TN + FP) > 0 else 0.0
        accuracy = TP / np.sum(cm[i, :]) if np.sum(cm[i, :]) > 0 else 0.0

        specificity_per_class.append(specificity)
        accuracy_per_class.append(accuracy)

    # 计算宏平均和微平均 Specificity
    macro_specificity = np.mean(specificity_per_class)
    micro_specificity = np.mean([TN / (TN + FP) if (TN + FP) > 0 else 0.0 for i in range(len(class_labels))])

    # 保存分类报告
    json_save_path = os.path.join(save_dir, "classification_report.json")
    with open(json_save_path, "w") as f:
        json.dump(classification_report_dict, f, indent=4)
    print(f"📌 分类报告已保存至: {json_save_path}")

    # 打印分类报告
    print("\n📊 分类报告:")
    print(f"{'类别':<20} {'Precision':<12} {'Recall':<12} {'F1-Score':<12} {'Specificity':<12} {'Accuracy':<12} {'Support':<12}")
    for idx, cls in enumerate(class_labels):
        metrics = classification_report_dict[cls]
        print(f"{cls:<20} {metrics['precision']:<12.2f} {metrics['recall']:<12.2f} {metrics['f1-score']:<12.2f} "
              f"{specificity_per_class[idx]:<12.2f} {accuracy_per_class[idx]:<12.2f} {metrics['support']:<12}")

    # 计算宏平均准确率
    macro_accuracy = np.mean(accuracy_per_class)

    # 输出微平均和宏平均指标
    print(f"\n📌 微平均准确率: {micro_accuracy:.4f}")
    print(f"📌 宏平均准确率: {macro_accuracy:.4f}")

    # 计算宏平均和微平均 Precision、Recall、F1-Score
    macro_precision = precision_score(true_labels, predicted_classes, average='macro', zero_division=0)
    macro_recall = recall_score(true_labels, predicted_classes, average='macro', zero_division=0)
    macro_f1 = f1_score(true_labels, predicted_classes, average='macro', zero_division=0)

    micro_precision = precision_score(true_labels, predicted_classes, average='micro', zero_division=0)
    micro_recall = recall_score(true_labels, predicted_classes, average='micro', zero_division=0)
    micro_f1 = f1_score(true_labels, predicted_classes, average='micro', zero_division=0)

    print("\n📊 宏平均指标:")
    print(f"Precision: {macro_precision:.4f}, Recall: {macro_recall:.4f}, F1-Score: {macro_f1:.4f}, Specificity: {macro_specificity:.4f}")

    print("\n📊 微平均指标:")
    print(f"Precision: {micro_precision:.4f}, Recall: {micro_recall:.4f}, F1-Score: {micro_f1:.4f}, Specificity: {micro_specificity:.4f}")

# ✅ 调用评估函数
evaluate_model(true_labels, predicted_classes, class_labels, save_dir)


In [ ]:
# Model Loading

In [ ]:
model = load_model('model_best.h5', custom_objects={'CustomLoss': CustomLoss})

In [ ]:
# Prediction and Inference

In [ ]:
# Make predictions on the test data
predictions = model.predict(test_generator)  
true_labels = test_generator.classes  
class_labels = list(test_generator.class_indices.keys())  
predicted_classes = np.argmax(predictions, axis=1)

In [ ]:
# Calculate confusion matrix
cm = confusion_matrix(true_labels, predicted_classes)

def evaluate_model(true_labels, predicted_classes, class_labels, save_dir):
    # Ensure save directory exists
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    # Micro average accuracy
    micro_accuracy = accuracy_score(true_labels, predicted_classes)
    
    # Generate classification report
    classification_report_dict = classification_report(
        true_labels, predicted_classes, target_names=class_labels, output_dict=True, zero_division=0
    )

    # Calculate specificity and accuracy per class
    specificity_per_class = []
    accuracy_per_class = []
    total_tn, total_fp = 0, 0  # For micro-average specificity
    for i in range(len(class_labels)):
        true_negative = np.sum(cm) - (np.sum(cm[i, :]) + np.sum(cm[:, i]) - cm[i, i])
        false_positive = np.sum(cm[:, i]) - cm[i, i]
        specificity = true_negative / (true_negative + false_positive) if (true_negative + false_positive) > 0 else 0.0
        specificity_per_class.append(specificity)

        accuracy = cm[i, i] / np.sum(cm[i, :]) if np.sum(cm[i, :]) > 0 else 0.0
        accuracy_per_class.append(accuracy)

        total_tn += true_negative
        total_fp += false_positive

    # Calculate macro and micro average specificity
    macro_specificity = np.mean(specificity_per_class)
    micro_specificity = total_tn / (total_tn + total_fp) if (total_tn + total_fp) > 0 else 0.0

    # Save classification report as JSON
    json_save_path = os.path.join(save_dir, "classification_report.json")
    with open(json_save_path, "w") as f:
        json.dump(classification_report_dict, f, indent=4)
    print(f"Classification Report saved to {json_save_path}")

    # Print classification report
    print("Classification Report:")
    print(f"{'Class':<20} {'Precision':<12} {'Recall':<12} {'F1-Score':<12} {'Specificity':<12} {'Accuracy':<12} {'Support':<12}")
    for idx, cls in enumerate(class_labels):
        metrics = classification_report_dict[cls]
        print(f"{cls:<20} {metrics['precision']:<12.2f} {metrics['recall']:<12.2f} {metrics['f1-score']:<12.2f} {specificity_per_class[idx]:<12.2f} {accuracy_per_class[idx]:<12.2f} {metrics['support']:<12}")

    # Calculate macro average accuracy
    macro_accuracy = np.mean(accuracy_per_class)

    # Print micro average accuracy
    print(f"\nMicro Average Accuracy: {micro_accuracy:.2f}")
    
    # Print macro average accuracy
    print(f"Macro Average Accuracy: {macro_accuracy:.2f}")

    # Macro and Micro average metrics
    macro_precision = precision_score(true_labels, predicted_classes, average='macro', zero_division=0)
    macro_recall = recall_score(true_labels, predicted_classes, average='macro', zero_division=0)
    macro_f1 = f1_score(true_labels, predicted_classes, average='macro', zero_division=0)

    micro_precision = precision_score(true_labels, predicted_classes, average='micro', zero_division=0)
    micro_recall = recall_score(true_labels, predicted_classes, average='micro', zero_division=0)
    micro_f1 = f1_score(true_labels, predicted_classes, average='micro', zero_division=0)

    print("\nMacro Average Metrics:")
    print(f"Precision: {macro_precision:.2f}, Recall: {macro_recall:.2f}, F1-Score: {macro_f1:.2f}, Specificity: {macro_specificity:.2f}")

    print("\nMicro Average Metrics:")
    print(f"Precision: {micro_precision:.2f}, Recall: {micro_recall:.2f}, F1-Score: {micro_f1:.2f}, Specificity: {micro_specificity:.2f}")

evaluate_model(true_labels, predicted_classes, class_labels, save_dir)


In [ ]:
#lime

In [ ]:
#gradcam

In [ ]:
#shape

In [ ]:
#ex

In [ ]:
import gc

K.clear_session()
gc.collect()
del model

In [ ]:
from numba import cuda

cuda.select_device(0)
cuda.close()